In [5]:
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
pip install torchsummary

Note: you may need to restart the kernel to use updated packages.


In [6]:
model = LeNet()

from torchsummary import summary
summary(model, input_size=(3, 32, 32),device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Linear-5                  [-1, 120]          48,120
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 62,006
Trainable params: 62,006
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.31
----------------------------------------------------------------


In [7]:
import torch
import torchvision
import torchvision.transforms as transforms

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [9]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=0)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
dir(torch)

['AVG',
 'AggregationType',
 'AliasDb',
 'AnyType',
 'Argument',
 'ArgumentSpec',
 'BFloat16Storage',
 'BFloat16Tensor',
 'BenchmarkConfig',
 'BenchmarkExecutionStats',
 'Block',
 'BoolStorage',
 'BoolTensor',
 'BoolType',
 'BufferDict',
 'ByteStorage',
 'ByteTensor',
 'CONV_BN_FUSION',
 'CallStack',
 'Callable',
 'Capsule',
 'CharStorage',
 'CharTensor',
 'ClassType',
 'Code',
 'CompilationUnit',
 'CompleteArgumentSpec',
 'ComplexDoubleStorage',
 'ComplexFloatStorage',
 'ComplexType',
 'ConcreteModuleType',
 'ConcreteModuleTypeBuilder',
 'DeepCopyMemoTable',
 'DeserializationStorageContext',
 'DeviceObjType',
 'DictType',
 'DisableTorchFunction',
 'DoubleStorage',
 'DoubleTensor',
 'EnumType',
 'ErrorReport',
 'ExecutionPlan',
 'FUSE_ADD_RELU',
 'FatalError',
 'FileCheck',
 'FloatStorage',
 'FloatTensor',
 'FloatType',
 'FunctionSchema',
 'Future',
 'FutureType',
 'Generator',
 'Gradient',
 'Graph',
 'GraphExecutorState',
 'HOIST_CONV_PACKED_PARAMS',
 'HalfStorage',
 'HalfTensor',
 'I

In [16]:
# Build GPU device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = LeNet()
# Place the model on the GPU
net.to(device)

LeNet(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [17]:
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [21]:
pip install tensorboard

     ---------------------------------------- 5.5/5.5 MB 2.7 MB/s eta 0:00:00
     ---------------------------------------- 4.4/4.4 MB 3.4 MB/s eta 0:00:00
     -------------------------------------- 431.6/431.6 kB 4.5 MB/s eta 0:00:00
     -------------------------------------- 133.7/133.7 kB 4.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [22]:
from torch.utils.tensorboard import SummaryWriter
# logs save the logs of training 10 epochs
writer = SummaryWriter('logs/logs')

In [23]:
# Start training logo
print('Start Training')

# Calculate the sum of loss, in order to calculate the average loss value
training_loss = 0.0
testing_loss = 0.0
for epoch in range(10):

    for data in trainloader:
    	# Put the data on the GPU
        inputs, labels = data[0].to(device), data[1].to(device)
		# Clear gradient information
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
		
		# Backpropagation, calculate gradient
        loss.backward()
        # Update one step parameters
        optimizer.step()
		# Calculate the loss value of this epoch and
        training_loss += loss.item()

	# torch.no_grad() is the context manager of PyTorch,
	# The test will not monitor the gradient, saving memory space
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)

            testing_loss += loss.item()


	# Use TensorBoard to draw a discount map to compare the loss on the training set and the test machine
    writer.add_scalars('Train-Test Loss',
                       {'Train Loss': training_loss / len(trainloader),
                        'Test Loss': testing_loss / len(testloader)}, epoch)
	
	# Clear the loss every epoch
    testing_loss = 0.0
    training_loss = 0.0
	
	# Print once after completing an epoch, you can master the training progress
    print("{} finished".format(epoch))
# Close TensorBoard
writer.close()

# Training completed sign
print('Finished Training')

Start Training
0 finished
1 finished
2 finished
3 finished
4 finished
5 finished
6 finished
7 finished
8 finished
9 finished
Finished Training


In [34]:
PATH = './cifar_net_10.pth'
torch.save(net.state_dict(), PATH)

In [35]:
import torch

# Instantiate the model
net = LeNet()
PATH = 'cifar_net_10.pth'
# Import the trained parameters
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [36]:
import torchvision.transforms as transforms

'''
 The input of the model we designed is 32*32, so when testing, you should still input a 32*32 size picture.
'''
transform = transforms.Compose(
    [transforms.Resize((32,32)),
     transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [37]:
from PIL import Image

# Load the picture as a PIL picture and store them as img1, img2, img3, img4
img0,img1,img2,img3 = map(Image.open,
                          ('data/dog.jpg','data/horse.jpg',
                           'data/ship.jpg','data/truck.jpg'))
                                                     
img0,img1,img2,img3 = map(transform, (img0,img1,img2,img3)) 

In [43]:
img0

tensor([[[[ 0.6000,  0.5922,  0.5608,  ...,  0.7098,  0.7020,  0.6549],
          [ 0.7098,  0.6863,  0.7020,  ...,  0.7020,  0.6392,  0.5216],
          [ 0.7647,  0.7882,  0.7882,  ...,  0.7176,  0.6314,  0.4196],
          ...,
          [ 0.0196,  0.1843,  0.2706,  ..., -0.1137, -0.4039, -0.6235],
          [-0.0667,  0.1216,  0.2235,  ..., -0.1451, -0.3882, -0.5529],
          [-0.1137,  0.0745,  0.1843,  ..., -0.3255, -0.3333, -0.4431]],

         [[ 0.4980,  0.4980,  0.4902,  ...,  0.7020,  0.6941,  0.6078],
          [ 0.6706,  0.6235,  0.6471,  ...,  0.6627,  0.6000,  0.4275],
          [ 0.7412,  0.7490,  0.7569,  ...,  0.6863,  0.5686,  0.2392],
          ...,
          [-0.3882, -0.2471, -0.1922,  ..., -0.2471, -0.4980, -0.7020],
          [-0.4431, -0.2941, -0.2392,  ..., -0.2706, -0.5137, -0.6706],
          [-0.4824, -0.3255, -0.2549,  ..., -0.4353, -0.4745, -0.5843]],

         [[-0.2000, -0.2235, -0.3020,  ..., -0.0196,  0.0745, -0.0353],
          [ 0.2392,  0.0588,  

In [38]:
# Add the dimension representing the batch size
img0 = torch.unsqueeze(img0, dim=0)
img1 = torch.unsqueeze(img1, dim=0)
img2 = torch.unsqueeze(img2, dim=0)
img3 = torch.unsqueeze(img3, dim=0)
# Four pictures into one batch
images = torch.cat((img0, img1, img2, img3),dim=0)

In [39]:
with torch.no_grad():
    outputs = net(images)
    # pred is the index value of the maximum value
    _, pred = torch.max(outputs,dim=1)
    pred = pred.numpy()

In [42]:
outputs

tensor([[-3.5457, -4.0126,  1.4534,  5.1587,  2.1258,  7.4939, -2.9982,  2.9126,
         -6.1049, -3.9813],
        [-3.5085, -4.5668,  0.7684,  0.2262,  5.5572,  2.9193, -4.2482,  6.4809,
         -5.4464,  1.4299],
        [ 5.6114,  2.4895, -1.5722, -3.5705, -3.5359, -4.3670, -0.5754, -5.7005,
          7.9634, -0.6917],
        [ 1.6218, -0.6797, -0.7607, -2.3429, -0.6468, -2.5042, -1.3180, -1.4051,
          4.7452,  3.8006]])

In [41]:
pred

array([5, 7, 8, 8], dtype=int64)

In [40]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

for i in range(4):
    print(classes[pred[i]])

dog
horse
ship
ship
